In [ ]:
import pandas as pd
import os
import pickle

In [ ]:
#Download the initial test statistics
df = pd.read_csv('Data/permutation_test_files_metabolomics/initial_tstats.csv', index_col=0)
#df = pd.read_csv('Data/permutation_test_files_proteomics/initial_tstats.csv', index_col=0)


### Reading in the files

Used to test the code I use for the HPC

In [ ]:
os.getcwd()

In [ ]:
#path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics'
#path = os.getcwd() + '\\Data\\permutation_test_files_proteomics'
path = os.getcwd() + '\\Data\\permutation_test_files_integrated'

Read all the files that start with 'Run' inside a folder:

In [ ]:
final_list = []

for filename in os.listdir(path): #also lists directories
    if filename.startswith('Run'):
        file_num = int(filename[3:-4])  #obtain the number of the Run
        if file_num > 11:
            print(filename)
            with open(os.path.join(path, filename), 'rb') as file: # open in readonly mode
                list1 = pickle.load(file)
            final_list.append(list1)

Read all the files in a folder that starts with 'Results':

In [ ]:
path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics'
final_list = []

for filename in os.listdir(path): #also lists directories
    if filename.startswith('Results'):
        print(filename)
        #for file in os.listdir(path + '\\' + filename): 
        for file in os.listdir(os.path.join(path,filename)):
            print(file)
            with open(os.path.join(path, filename,file), 'rb') as file: # open in readonly mode
                list1 = pickle.load(file)
            final_list.append(list1)

In a folder with directories list (Results1, Results2, Results3 etc) go through each as a separate array job:


In [ ]:
#index_num = sys.argv[1]  #this should return the array number within the array job
index_num = 1 #for example
path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics\\Results' + str(index_num)
final_list = []

for filename in os.listdir(path): #also lists directories
    print(filename)
    with open(os.path.join(path, filename), 'rb') as file: # open in readonly mode
        list1 = pickle.load(file)
    final_list.append(list1)

In [ ]:
len(final_list)

for list1 in final_list:
    print(len(list1))
    print(list1[:3])

### Calculate which edges are significant

Calculates number of permuted values above the initial test statistic, then divides by number of tests and works out p-value, before outputting the significant edges: (NOT RECOMMENDED, p-values not FDR corrected)

In [ ]:
sig_edges = []
edgelist = df.index

for index,pathway_pair in enumerate(edgelist):   #test all pathways
    comparison = df.Initial_tstat[index]    #get initial test statistic
    counter = 0
    
    for list1 in final_list:  #len(final_list) = number of permutations
        if abs(list1[index]) >= comparison:   
            counter += 1
    
    p_val = (counter/len(final_list))    #divide number of tests above or equal to the test statistic by total number of tests

    if p_val < 0.01:  #for 100,000 permutations
        sig_edges.append(pathway_pair)  
        
print(len(sig_edges))
sig_edges

Calculates number of permuted values above the initial test statistic (more scalable)

In [ ]:
num_vals = []  #number of permuted values above the magnitude of the test statistic
#sig_edges = []
edgelist = df.index

for index in range(len(edgelist)):   #test all pathways
    comparison = df.Initial_tstat[index]    #get initial test statistic
    print(comparison)
    counter = 0
    
    for list1 in final_list:  #len(final_list) = number of permutations
        if abs(list1[index]) >= comparison:   
            counter += 1
    num_vals.append(counter)

num_vals

### Another alternative: (but converting to numpy array actually seems to take up more memory)

In [ ]:
import numpy as np

final_matrix = np.empty((0,len(df)), int)

for filename in os.listdir(path):
    if filename != 'initial_tstats.csv':
        with open(os.path.join(path, filename), 'rb') as file: # open in readonly mode
            list1 = pickle.load(file)
        final_matrix = np.append(final_matrix, np.array([list1]), axis=0)

In [ ]:
final_matrix.shape

In [ ]:
final_matrix = np.array(final_list) 

final_matrix.shape

sig_edges = []
edgelist = df.index

for index,pathway_pair in enumerate(edgelist):   #test all pathways
    comparison = df.Initial_tstat[index]    #get initial test statistic
    counter = 0
    
    col = abs(final_matrix[:,index])  #index a column and turn into absolute values
    counter = (col >= comparison).sum()

    
    p_val = (counter/len(final_list))    #divide number of tests above or equal to the test statistic by total number of tests

    if p_val < 0.01:  #for 100,000 permutations
        sig_edges.append(pathway_pair)  
        
print(len(sig_edges))

In [ ]:
#comparing the size of the object from the first method vs second method ()
from sys import getsizeof
print(getsizeof(final_list))
print(getsizeof(final_matrix))

Saving the file:

In [ ]:
#Writing as text file
with open ('Data/permutation_test_files_metabolomics/vals_test.txt', 'w') as file:
     file.write(','.join(str(i) for i in num_vals))

In [ ]:
#Pickling the file to save on memory space
with open('Data/permutation_test_files_metabolomics/vals_test.txt', "wb") as file:  
       pickle.dump(output,file)

### Read in HPC results (reading ten files):

In [ ]:
import os 
import numpy as np 
import statsmodels.stats.multitest
import pandas as pd
from itertools import compress

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Read ten files that have the number of edges that are of a higher value that the initial test statistic

#path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics\\Values'
#path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics_withoutPA\\Values'
path = os.getcwd() + '\\Data\\permutation_test_files_proteomics\\Values'
#path = os.getcwd() + '\\Data\\permutation_test_files_integrated\\Values'
val_array = []


for filename in os.listdir(path): #also lists directories
    with open(os.path.join(path, filename)) as file:    
        lines = file.readlines()
        vals = lines[0].split(';')
        vals =  [int(x) for x in vals]
        #print(vals)
        if filename == 'vals1.txt':
            val_array = np.array(vals)
        else:
            val_array = np.vstack([val_array, vals])

In [ ]:
val_array
val_array2 = val_array.sum(axis=0) #add up the values by columns for each pathway pair
pval_array = val_array2 / 100000
pval_array[:20]

print(len(pval_array)) #10,296 for metabolomic,55,278 for metabolomic_withoutPA, 166,753 values for proteomic, 251,695 values for integrated
(pval_array < 0.05).sum() #763 for metabolomic, 3933 for metabolomic_withoutPA, 58,115 values for proteomic, 68,299 values for integrated

#Before I realised I forgot to scale: 795 values for metabolomic, 56,256 values for proteomic, 65,208 values for integrated

In [ ]:
sns.histplot(pval_array, bins = 50,color='#B8CDF8',edgecolor="k") 

#plt.title('Metabolomic unadjusted p-value distribution',fontsize=16)
#plt.title('Metabolomic without PA unadjusted p-value distribution',fontsize=16)
#plt.title('Proteomic unadjusted p-value distribution',fontsize=16)
#plt.title('Integrated unadjusted p-value distribution',fontsize=16)

plt.xlabel('Unadjusted p-value',fontsize=13)
plt.ylabel('Count',fontsize=13) 


#plt.savefig( 'Figures/unadjusted_pval_metabolomic.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')
#plt.savefig( 'Figures/unadjusted_pval_metabolomic_withoutPA.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')
#plt.savefig( 'Figures/unadjusted_pval_proteomic.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')
#plt.savefig( 'Figures/unadjusted_pval_integrated.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

In [ ]:
sig_edge_boolean,adjusted_pval = statsmodels.stats.multitest.fdrcorrection(pval_array, alpha=0.005, method='poscorr', is_sorted=False)
sig_edge_boolean

In [ ]:
sig_edge_boolean2,adjusted_pval2 = statsmodels.stats.multitest.fdrcorrection(pval_array, alpha=0.005, method='indep', is_sorted=False)
sig_edge_boolean2

In [ ]:
print(adjusted_pval)
print(adjusted_pval2)

In [ ]:
sig_edge_boolean == sig_edge_boolean2
np.array_equiv(sig_edge_boolean,sig_edge_boolean2)

In [ ]:
adjusted_pval

In [ ]:
(adjusted_pval < 0.005).sum()

In [ ]:
significance_ordered = list(adjusted_pval[adjusted_pval < 0.005])
significance_ordered.sort()
significance_ordered

In [ ]:
sum(sig_edge_boolean) 

#One-sided permutation testing:
#Metabolomic - 214 edges at p < 0.005 and 53 edges at  p < 0.005
#Proteomic - 15,730 edges at p < 0.05 and 8,753 edges at p < 0.005
#Integrated - 45,123 edges at p < 0.05 and 9,069 edges at p < 0.005

#Two sided permutation testing (before I realised that I forgot to scale the two dataset after subsetting)
#Metabolomic - 8 edges significant with method=indep and method=poscorr
#Metabolomic without PA - 70 edges significant with method=indep and method=poscorr
#Proteomic - 31,486 edges significant with method=indep and method=poscorr
#Integrated - 26,761 edges significant with method=indep and method=poscorr (less than proteomic because more edges so penalised more heavily)


In [ ]:
sns.histplot(adjusted_pval, bins = 50,color='#B8CDF8',edgecolor="k") 

#plt.title('Metabolomic adjusted p-value distribution',fontsize=16)
#plt.title('Metabolomic without PA adjusted p-value distribution',fontsize=16)
plt.title('Proteomic adjusted p-value distribution',fontsize=16)
#plt.title('Integrated adjusted p-value distribution',fontsize=16)

plt.xlabel('Adjusted p-value',fontsize=13)
plt.ylabel('Count',fontsize=13) 

#plt.savefig( 'Figures/adjusted_pval_metabolomic.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')
#plt.savefig( 'Figures/adjusted_pval_metabolomic_withoutPA.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')
#plt.savefig( 'Figures/adjusted_pval_proteomic.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')
#plt.savefig( 'Figures/adjusted_pval_integrated.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')



Easier to obtain the reverse (i.e. edges that are not significant), since then they can be removed easily

In [ ]:
not_sig_edge_boolean = [not elem for elem in sig_edge_boolean]
sum(not_sig_edge_boolean )

In [ ]:
#Load dataset
#df = pd.read_csv('Data/permutation_test_files_metabolomics/initial_tstats.csv', index_col=0)
#df = pd.read_csv('Data/permutation_test_files_metabolomics_withoutPA/initial_tstats.csv', index_col=0)
#df = pd.read_csv('Data/permutation_test_files_proteomics/initial_tstats.csv', index_col=0)
df = pd.read_csv('Data/permutation_test_files_integrated/initial_tstats.csv', index_col=0)
edgelist = df.index


sig_edges = list(compress(edgelist,sig_edge_boolean))
not_sig_edges = list(compress(edgelist,not_sig_edge_boolean))

#Zip p-values to edge names to form dictionary (so I can test significance of differentially expressed genes)
edgelist_tuple = []
for i in range(len(edgelist)):
    edge = edgelist[i].split(",")
    edge = (edge[0],edge[1][1:])
    edgelist_tuple.append(edge)
    
p_val_dict  = dict(zip(edgelist_tuple, adjusted_pval))

Saving the significant edges: Easier to compare with the naive networks and the other omics networks as well <br>
Saving the non-significant edges: Easier when constructing the differential network analysis

In [ ]:
#with open ('Data/permutation_test_files_metabolomics/sig_edges.txt', 'w') as file:
#     file.write(','.join(str(i) for i in sig_edges))
     
#with open ('Data/permutation_test_files_metabolomics/not_sig_edges.txt', 'w') as file:
#     file.write(','.join(str(i) for i in not_sig_edges))

In [ ]:
#Save number of significant edges BEFORE FDR CORRECTION

#sig_edges_beforefdr = list(compress(edgelist,pval_array < 0.05))
#print(len(sig_edges_beforefdr))

#with open ('Data/permutation_test_files_metabolomics/sig_edges_beforefdr.txt', 'w') as file:
#     file.write(','.join(str(i) for i in sig_edges_beforefdr))

In [ ]:
#with open ('Data/permutation_test_files_metabolomics_withoutPA/sig_edges.txt', 'w') as file:
#     file.write(','.join(str(i) for i in sig_edges))
     
#with open ('Data/permutation_test_files_metabolomics_withoutPA/not_sig_edges.txt', 'w') as file:
#     file.write(','.join(str(i) for i in not_sig_edges))

In [ ]:
#with open ('Data/permutation_test_files_proteomics/sig_edges.txt', 'w') as file:
#     file.write(','.join(str(i) for i in sig_edges))
     
#with open ('Data/permutation_test_files_proteomics/not_sig_edges.txt', 'w') as file:
#     file.write(','.join(str(i) for i in not_sig_edges))

In [ ]:
#with open ('Data/permutation_test_files_integrated/sig_edges.txt', 'w') as file:
#     file.write(','.join(str(i) for i in sig_edges))
     
#with open ('Data/permutation_test_files_integrated/not_sig_edges.txt', 'w') as file:
#     file.write(','.join(str(i) for i in not_sig_edges))

### Plotting the test statistic distribution for randomly chosen pathway pairs


In [ ]:
import pandas as pd
import os
import random
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Download the initial test statistics
#df = pd.read_csv('Data/permutation_test_files_metabolomics/initial_tstats.csv', index_col=0)
#df = pd.read_csv('Data/permutation_test_files_proteomics/initial_tstats.csv', index_col=0)
df = pd.read_csv('Data/permutation_test_files_integrated/initial_tstats.csv', index_col=0)


#path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics'
#path = os.getcwd() + '\\Data\\permutation_test_files_proteomics'
path = os.getcwd() + '\\Data\\permutation_test_files_integrated'

In [ ]:
df

Selecting random pathways and writing to a file without the HPC:

In [ ]:


#pathway_list = []

#for i in range(8): #take 8 random pathways
#    index = random.randrange(0, len(df)) #Gives index from 0 to (len(df)-1), better for indexing with unpickled files which are stored as lists
#    print(index)
#    for filename in os.listdir(path):
#            if filename.startswith('Run'):
#                with open(os.path.join(path, filename),'rb') as file:
#                    list1 = pickle.load(file)   
#                    #print(len(list1))
#                    pathway_list.append(list1[index])
#    with open('Data/permutation_test_files_metabolomics/test_distribution'+str(index)+'.txt', 'w') as file:
#        file.write(','.join(str(i) for i in pathway_list))
#    print(len(pathway_list))
#    pathway_list = []




Read in the test distribution files that were made from the HPC and plotting:

In [ ]:
val_array = []


#Run all the test distribution files:
#for filename in os.listdir(path): #also lists directories
#    if filename.startswith('test'):
#        with open(os.path.join(path, filename)) as file:    
#            lines = file.readlines()
#            vals = lines[0].split(',')
#            vals =  [float(x) for x in vals]
#            print(len(vals))
#            print(vals[:100])
            

#I've plotted figures for the following (metabolomics): 855, 1496 (sig before correction), 1884, 1897, 2391, 2681 (sig before correction, not normal), 2747, 3660, 4665, 4692, 5958, 7664 (normal), 8257, 9949, 10215
#I've plotted figures for the following (proteomics): 492 (sig - no permutations over), 1863 (weird bimodal distribution, 3212 (sig before correction), 3379, 11886 (sig before correction), 15302 (sig before correction), 19587 (sig before correction, weird distribution), 44413 (sig), 80764, 98000(sig), 142402, 143312, 145571 (sig, weird distribution), 153715 (sig - no permutations over), 157122 (not sig, normal)
#I've plotted figures for the following (integrated): 19156, 29535, 39570, 67801 (sig before correction), 82140, 96565, 96908 (sig before correction, normal), 101041, 124135, 184762 (sig), 236500 (sig before correction)

#Open the test distribution files one by one:
test_num = 239468
#with open('Data/permutation_test_files_metabolomics/Test_distribution/test_distribution'+str(test_num)+'.txt') as file:
#with open('Data/permutation_test_files_proteomics/Test_distribution/test_distribution'+str(test_num)+'.txt') as file:
with open('Data/permutation_test_files_integrated/Test_distribution/test_distribution'+str(test_num)+'.txt') as file:
    comparison = df.Initial_tstat[test_num]
    edge_name = df.index[test_num]

    lines = file.readlines()
    vals = lines[0].split(',')
    vals =  [float(x) for x in vals]
    print(len(vals))
    print(vals[:100])


#For metabolomics:
#sns.histplot(vals, bins = 50,color='#59C3C3',edgecolor="k") 

#For proteomics:
#sns.histplot(vals, bins = 50,color='#4062BB',edgecolor="k") 

#For integrated data:
sns.histplot(vals, bins = 50,color='#52489C',edgecolor="k") 


plt.title('Pathway pair '+ edge_name,fontsize=16)
plt.xlabel('Delta value',fontsize=13)
plt.ylabel('Count',fontsize=13) 

plt.axvline(x=comparison, color='r', linewidth=0.5, linestyle='--', dashes=(8, 5))
plt.axvline(x=-comparison, color='r', linewidth=0.5, linestyle='--', dashes=(8, 5))

#Count number of values (made absolute) that are over the initial test statistic (made absolute) for two sided permutation test
counter = 0 
for index in range(len(vals)):  #len(final_list) = number of permutations
    if abs(vals[index]) >= comparison:   
        counter += 1
print(counter)


#MUST RUN THE ABOVE CODE TO LOAD IN THE NUMBER OF VALUES THAT ARE ABOVE THE INITIAL TEST STATISTIC SO I KNOW WHETHER THE PATHWAY PAIR IS SIGNIFICANT AFTER FDR CORRECTION
if sig_edge_boolean[test_num] == True:
    plt.text(-0.8, -2600, "Adjusted p-value significant",fontsize=10)
else:
    plt.text(-0.8, -2600, "Not significant",fontsize=10)
plt.text(-0.8, -3200, "Unadjusted p-value: " + str(pval_array[test_num]),fontsize=10)
plt.text(-0.8, -3800, "FDR-adjusted p-value: " + str(adjusted_pval[test_num]),fontsize=10)
plt.text(-0.8, -4400, "Number of values above initial test statistic: " + str(counter),fontsize=10)



plt.xlim(-0.9,0.9)   #-log_10(0.005) = 2.301...
plt.ylim(0,15000) 

#plt.savefig( 'Figures/Pathway ' + edge_name + '.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

### Comparing the difference in edges with other networks

Comparing with the naive networks (mild vs severe) to see if they agree with the differential network results:

In [ ]:
import networkx as nx

In [ ]:
mild = nx.read_gml("Cytoscape/metabolomic_final_mildcases.gml")
severe = nx.read_gml("Cytoscape/metabolomic_final_severecases.gml")

In [ ]:
mild = nx.read_gml("Cytoscape/proteomic_final_mildcases.gml")
severe = nx.read_gml("Cytoscape/proteomic_final_severecases.gml")

In [ ]:
mild = nx.read_gml("Cytoscape/integrated_final_mildcases.gml")
severe = nx.read_gml("Cytoscape/integrated_final_severecases.gml")

In [ ]:
mild_edges = list(mild.edges())
print(len(mild_edges))
severe_edges = list(severe.edges())
print(len(severe_edges))

In [ ]:
#Example code from https://stackoverflow.com/questions/41125909/python-find-elements-in-one-list-that-are-not-in-the-other
list_1=["a", "b", "c", "d", "e"]
list_2=["a", "f", "c", "m"]
set(list_2) - set(list_1)

#set(['m', 'f'])

In [ ]:
#Edges present in the mild network but not the severe network
print(len(list(set(mild_edges) - set(severe_edges))))
mild = list(set(mild_edges) - set(severe_edges))

In [ ]:
#Edges present in the severe network but not the mild network
print(len(list(set(severe_edges) - set(mild_edges))))
severe = list(set(severe_edges) - set(mild_edges))

In [ ]:
#Edges present in both mild and severe
print(len(list(set(severe_edges) & set(mild_edges))))
mild_severe = list(set(severe_edges) & set(mild_edges))

In [ ]:
#Reading in the edges expressed in the differential metabolomic network
#with open('Data/permutation_test_files_metabolomics/sig_edges_beforefdr.txt') as f:
#with open('Data/permutation_test_files_metabolomics/sig_edges.txt') as f:
#with open('Data/permutation_test_files_proteomics/sig_edges.txt') as f:
with open('Data/permutation_test_files_integrated/sig_edges.txt') as f:
    lines = f.readlines()

edges_remaining = []

edges = lines[0].split(",")

for index in range(0,len(edges),2):
    list1 = edges[index],(edges[index+1][1:]) #becomes a tuple
    edges_remaining.append(list1)

print(len(edges_remaining))
#edges_remaining

In [ ]:
intersection = list(set(edges_remaining).intersection(list(set(mild))))  
len(intersection)

In [ ]:
intersection = list(set(edges_remaining).intersection(list(set(severe)))) 
len(intersection) 

In [ ]:
intersection = list(set(edges_remaining).intersection(list(set(mild_severe)))) 
len(intersection) 

### Comparing with the significant edges BETWEEN the different omics:

In [ ]:
def edge_num (omics):
    with open('Data/permutation_test_files_'+omics+'/sig_edges.txt') as f:
    #with open('Data/permutation_test_files_'+omics+'/sig_edges_beforefdr.txt') as f:    #for metabolomic data since there are so few edges left
        lines = f.readlines()
    edges_remaining = []
    edges = lines[0].split(",")

    for index in range(0,len(edges),2):
        list1 = edges[index],(edges[index+1][1:]) #becomes a tuple
        edges_remaining.append(list1)

    return edges_remaining


In [ ]:
proteomic_edges = edge_num('proteomics')
integrated_edges = edge_num('integrated')
metabolomic_edges = edge_num('metabolomics')

In [ ]:
len(integrated_edges)

In [ ]:
intersection = list(set(integrated_edges).intersection(list(set(metabolomic_edges))))  
len(intersection) 

In [ ]:
intersection = (set(metabolomic_edges) & set(proteomic_edges) & set(integrated_edges))  
len(intersection) 

In [ ]:
len(list(set(integrated_edges) - (set(metabolomic_edges)|set(proteomic_edges)))) #Pathway pairs for integrated data not detected by either single omics

### Get number of common pathways between the three omics

In [ ]:
import pandas as pd
import sspa

In [ ]:
metabolomic_df = pd.read_csv('Data/Su_COVID_metabolomics_processed_commoncases.csv', index_col=0)
proteomic_df = pd.read_csv('Data/Su_COVID_proteomics_processed_commoncases.csv', index_col=0)
integrated_df = pd.read_csv("Data/Su_integrated_data.csv", index_col=0)

#Remove root pathways
#Convert pathway ID to name
root_path = pd.read_excel('Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}
root_pathway_names = list(root_pathway_dict.keys())

In [ ]:
reactome_pathways = sspa.process_gmt("Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")
kpca_scores = sspa.sspa_kpca(metabolomic_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

metabolomic_pathways = kpca_scores.columns
metabolomic_pathways

In [ ]:
#Load in a file downloaded from https://reactome.org/download/current/UniProt2Reactome_All_Levels.txt
reactome_pathways = sspa.process_reactome('Homo sapiens', infile = 'Data/UniProt2Reactome_All_Levels_ver84.txt', download_latest = False, filepath = None)
kpca_scores = sspa.sspa_kpca(proteomic_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

proteomic_pathways = kpca_scores.columns
proteomic_pathways

In [ ]:
reactome_pathways = pd.read_csv("Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")
kpca_scores = sspa.sspa_kpca(integrated_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

integrated_pathways = kpca_scores.columns
integrated_pathways

In [ ]:
intersection = list(set(metabolomic_pathways).intersection(list(set(proteomic_pathways))))   #47 pathways
intersection = (set(metabolomic_pathways) & set(proteomic_pathways) & set(integrated_pathways))  #47 pathways 

intersection = list(set(metabolomic_pathways).intersection(list(set(integrated_pathways))))  #144 pathways (same as number of metabolomic)
intersection = list(set(proteomic_pathways).intersection(list(set(integrated_pathways))))  #578 pathways (same as number of proteomic)

#710 pathways in integrated dataset
#((144+578)-47+35)

len(list(set(integrated_pathways) - (set(metabolomic_pathways)|set(proteomic_pathways)))) #35 pathways for integrated data not detected by either single omics

In [ ]:
new_pathways = list(set(integrated_pathways) - (set(metabolomic_pathways)|set(proteomic_pathways)))
new_pathways

In [ ]:
#Get number of pathways that have been influenced

df = pd.read_csv("Data/Su_integrated_data.csv", index_col=0)
reactome_pathways = pd.read_csv("Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")


kpca_scores = sspa.sspa_kpca(integrated_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

#Obtain pathways and corresponding metabolites for all Reactome pathways, store as dictionary
orig_dict = sspa.utils.pathwaydf_to_dict(reactome_pathways)

#Filter out dictionary to retain only the pathways that remain after kPCA
my_keys = kpca_scores.columns
pathways_dict = {key: orig_dict[key] for key in my_keys}

#Filter out the compounds in the pathways that are not present in the dataset

#Obtain all unique values in dataset
compounds_present = list(df.columns[:-2])
filtered_dict = {} 

#My code adapted from Cecilia's
#If the key values are not part of the compounds in dataset then remove
for key,value in pathways_dict.items():
    new_val = [item for item in value if item in compounds_present]
    if len(new_val) >= 2: #at least two compounds in the pathway
        filtered_dict[key] = new_val

In [ ]:
new_pathways

['R-HSA-9734735',
 'R-HSA-196807',
 'R-HSA-379398',
 'R-HSA-901042',
 'R-HSA-2672351',
 'R-HSA-193681',
 'R-HSA-1483249',
 'R-HSA-73942',
 'R-HSA-202131',
 'R-HSA-8868773',
 'R-HSA-8978934',
 'R-HSA-203615',
 'R-HSA-373760',
 'R-HSA-975576',
 'R-HSA-9634815',
 'R-HSA-379401',
 'R-HSA-70350',
 'R-HSA-9660821',
 'R-HSA-983712',
 'R-HSA-209968',
 'R-HSA-9694631',
 'R-HSA-5652084',
 'R-HSA-5579029',
 'R-HSA-379397',
 'R-HSA-187015',
 'R-HSA-975578',
 'R-HSA-8939902',
 'R-HSA-1483166',
 'R-HSA-71240',
 'R-HSA-194002',
 'R-HSA-532668',
 'R-HSA-2162123',
 'R-HSA-389661',
 'R-HSA-9755088',
 'R-HSA-193993']

In [ ]:
#All 35 new pathways comprised of one metabolite and one protein

proteomic_only_pathway = []
proteomic_pathway_with_onemetabolite = []
metabolomic_only_pathway = []
metabolomic_pathway_with_oneprotein = []
both_omic = []
new_pathway = []

for key,values in filtered_dict.items():
    #if key in new_pathways:
        #print(values)
    #print(values)
    proteomic=False
    metabolomic=False
    both=False


    proteins = 0
    metabolites = 0

    for value in values:

        if value[0].isalpha():
            proteins += 1
        else:
            metabolites += 1

    
    if metabolites == 0:
        proteomic_only_pathway.append(key)
    if proteins > 1 and metabolites == 1:
        proteomic_pathway_with_onemetabolite.append(key)
    
    if proteins == 0:
        metabolomic_only_pathway.append(key)
    if metabolites > 1 and proteins == 1:
        metabolomic_pathway_with_oneprotein.append(key)

    if metabolites > 1 and proteins > 1:
        both_omic.append(key)
    if proteins == 1 and metabolites == 1:
        new_pathway.append(key)



In [ ]:
print(len(proteomic_only_pathway))
print(len(proteomic_pathway_with_onemetabolite))
print(len(metabolomic_only_pathway))
print(len(metabolomic_pathway_with_oneprotein))
print(len(both_omic))
print(len(new_pathway))

485
46
65
32
47
35


In [ ]:
for key,values in filtered_dict.items():
    #if key in new_pathways:
        #print(values)
    print(values)
    proteomic=False
    metabolomic=False
    both=False


    for value in values:

        if value[0].isalpha() and metabolomic==False:
            proteomic=True
        elif value[0].isdigit() and proteomic==False:
            metabolomic=True

        elif value[0].isalpha() and metabolomic==True:
            both=True
            metabolomic=False
            break
        elif value[0].isdigit() and proteomic==True:
            both=True
            proteomic=False
            break

    if proteomic == True:
        proteomic_only.append(key)
    if metabolomic == True:
        metabolomic_only.append(key)
    if both == True:
        both_omic.append(key)


### Comparing the initial test statistics of the three omics to see which edges were affected

In [ ]:
import pandas as pd

Comparing the initial test statistics of the integrated data with the proteomic initial test statistics:

In [ ]:
#Load dataset
metabolomic = pd.read_csv('Data/permutation_test_files_metabolomics/initial_tstats.csv', index_col=0)
proteomic = pd.read_csv('Data/permutation_test_files_proteomics/initial_tstats.csv', index_col=0)
integrated = pd.read_csv('Data/permutation_test_files_integrated/initial_tstats.csv', index_col=0)

In [ ]:
metabolomic = metabolomic.rename(columns={'Initial_tstat': 'metabolomic_tstat'})
proteomic = proteomic.rename(columns={'Initial_tstat': 'proteomic_tstat'})
integrated = integrated.rename(columns={'Initial_tstat': 'integrated_tstat'})

In [ ]:
initial_tstat_comp = proteomic.join(integrated)
initial_tstat_comp

In [ ]:
#Number of pathway pairs with initial test statistic that are the same

proteomic_only_edge = []
metabolomic_influence_edge = []
counter = 0

for edge in initial_tstat_comp.index:
    if initial_tstat_comp.loc[edge].proteomic_tstat == initial_tstat_comp.loc[edge].integrated_tstat:
        proteomic_only_edge.append(edge)
        counter += 1
    else:
        metabolomic_influence_edge.append(edge)


print(counter)

#166,753 edges of the proteomic initial test statistic dataset are all included in the integrated initial test statistic data
#117,381 edges out of 166,753 are identical
#Therefore 49,372 edges were influenced by the metabolomic data

len(metabolomic_influence_edge)

In [ ]:
#Change to tuple format so you can compare with the significant edges

metabolomic_influence_edge_tuple = []
for i in range(len(metabolomic_influence_edge)):
    edge = metabolomic_influence_edge[i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    metabolomic_influence_edge_tuple.append(edge)


proteomic_only_edge_tuple  = []
for i in range(len(proteomic_only_edge )):
    edge = proteomic_only_edge [i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    proteomic_only_edge_tuple.append(edge)

In [ ]:
integrated_edge = integrated.index

integrated_edge_tuple  = []
for i in range(len(integrated_edge)):
    edge = integrated_edge [i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    integrated_edge_tuple.append(edge)



In [123]:
#Need to see proportion of metabolomic-influenced edges that are significantly differentially expressed edges (from differential network) in the proteomic data vs the integrated data
print(len(metabolomic_influence_edge))
print(len(list(set(proteomic_edges).intersection(set(metabolomic_influence_edge_tuple))))) #2786 edges significant
print(len(list(set(integrated_edges).intersection(set(metabolomic_influence_edge_tuple))))) #After integrating with the metabolomic data, 2425 significant


#1438 edges are the same between 2786 edges and 2425 edges
list1 = list(set(proteomic_edges).intersection(set(metabolomic_influence_edge_tuple)))
list2 = list(set(integrated_edges).intersection(set(metabolomic_influence_edge_tuple)))
len(list(set(list1).intersection(set(list2))))

49372
2786
2425


1438

In [ ]:
#Showing that 'Metabolomic influenced' proteomic pathways are instances where at least one pathway is a common pathway OR one pathway is influenced by one metabolite (not enough to be considered a pathway in the metabolomic set)

list1 = list(set(proteomic_edges).intersection(set(metabolomic_influence_edge_tuple)))

common_pathways = list(set(proteomic_pathways) & set(metabolomic_pathways))
print(len(common_pathways))
counter = 0 

influenced_noncommon_pathways = []
for edges in list1:
    pathway_pair = list(edges)
    if (pathway_pair[0] in common_pathways) or (pathway_pair[1] in common_pathways):
        #print(pathway_pair)
        counter += 1
    else:
        influenced_noncommon_pathways.append(edges)


print(counter)
#1371 pathway pairs have one common pathway in them, out of 49,372 metabolomic influenced protein pathways
#The other cases are where there is one metabolite added to one of the pathways that is not enough to be constituted as a pathway in the metabolomics dataset

In [ ]:
df = pd.read_csv('Data/Su_COVID_metabolomics_processed_commoncases.csv', index_col=0)
reactome_pathways = sspa.process_gmt("Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")

#df = pd.read_csv("Data/Su_integrated_data.csv", index_col=0)
#reactome_pathways = pd.read_csv("Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")


kpca_scores = sspa.sspa_kpca(integrated_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

#Obtain pathways and corresponding metabolites for all Reactome pathways, store as dictionary
orig_dict = sspa.utils.pathwaydf_to_dict(reactome_pathways)

#Filter out dictionary to retain only the pathways that remain after kPCA
my_keys = kpca_scores.columns
pathways_dict = {key: orig_dict[key] for key in my_keys}

#Filter out the compounds in the pathways that are not present in the dataset

#Obtain all unique values in dataset
compounds_present = list(df.columns[:-2])
filtered_dict = {} 

#My code adapted from Cecilia's
#If the key values are not part of the compounds in dataset then remove
for key,value in pathways_dict.items():
    new_val = [item for item in value if item in compounds_present]
    if len(new_val) >= 2: #at least two compounds in the pathway
        filtered_dict[key] = new_val

In [ ]:
print(influenced_noncommon_pathways)
list(reactome_pathways.loc['R-HSA-422475'])
filtered_dict['R-HSA-422475']

Comparing the initial test statistics of the integrated data with the metabolomic initial test statistics:

In [ ]:
initial_tstat_comp = metabolomic.join(integrated)
initial_tstat_comp

In [ ]:
#Number of pathway pairs with initial test statistic that are the same
metabolomic_only_edge = []
proteomic_influence_edge = []

counter = 0
for edge in initial_tstat_comp.index:
    if initial_tstat_comp.loc[edge].metabolomic_tstat == initial_tstat_comp.loc[edge].integrated_tstat:
        metabolomic_only_edge.append(edge)
        counter += 1
    else:
        proteomic_influence_edge.append(edge)

print(counter)

#10,296 edges of the metabolomic initial test statistic dataset are all included in the integrated initial test statistic data
#2,088 edges out of 10,296 are identical
#Therefore 8,208 edges were influenced by the proteomic data

proteomic_influence_edge

In [ ]:
proteomic_influence_edge_tuple = []
for i in range(len(proteomic_influence_edge)):
    edge = proteomic_influence_edge[i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    proteomic_influence_edge_tuple.append(edge)


metabolomic_only_edge_tuple = []
for i in range(len(metabolomic_only_edge)):
    edge = metabolomic_only_edge[i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    metabolomic_only_edge_tuple.append(edge)

In [ ]:
#Need to see proportion of proteomic-influenced edges that are significant edges in the metabolomic data vs the integrated data
print(len(proteomic_influence_edge))
print(len(list(set(metabolomic_edges).intersection(set(proteomic_influence_edge_tuple))))) #43 edges significant
print(len(list(set(integrated_edges).intersection(set(proteomic_influence_edge_tuple))))) #After integrating with the proteomic data, 170 significant

#8 edges are the same between 43 edges and 170 edges
list1 = list(set(metabolomic_edges).intersection(set(proteomic_influence_edge_tuple)))
list2 = list(set(integrated_edges).intersection(set(proteomic_influence_edge_tuple)))
len(list(set(list1).intersection(set(list2))))

### Testing the integrated only pathways

In [129]:
integrated_only = list(set(integrated_edges) - (set(metabolomic_edges)|set(proteomic_edges)))
len(integrated_only)

2535

Common pathway | Common pathway            =   47   <br>

Common pathway | Metabolite only pathway      =   81   <br>
Proteomic-influenced metabolomic pathway | (Any)  =  26    <br>

Common pathway | Protein only edge   =   668   <br>
Proteomic-influenced metabolomic pathway | (Any)  =  272    <br>

Pathway only in metabolomics | Pathway only in proteomics   =   676   <br>

False positives: <br>
Pathway only in metabolomics | Pathway only in metabolomics   =   95  <br>
Pathway only in metabolomics | Pathway only in metabolomics   =   95  <br>

New integrated pathways: <br>
New pathway | (Any)   =   669  <br>


In [184]:
print(len(proteomic_only_pathway))
print(len(proteomic_pathway_with_onemetabolite))
print(len(metabolomic_only_pathway))
print(len(metabolomic_pathway_with_oneprotein))
print(len(both_omic))
print(len(new_pathway))

485
46
65
32
47
35


In [124]:
#Pathways where both are common pathways

common_pathways = list(set(proteomic_pathways) & set(metabolomic_pathways))
len(common_pathways)

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in common_pathways) and (pathway_pair[1] in common_pathways):
        #print(pathway_pair)
        counter += 1

print(counter)

47


In [125]:
#Proteomic influenced metabolomic pair (i.e. pairs with one of the 47 common pathways and one metabolomic only pair)
print(len(list(set(proteomic_influence_edge_tuple).intersection(set(integrated_only)))))
#Metabolomic influenced proteomic pair (i.e. pairs with one of the 47 common pathways and one proteomic only pair)
print(len(list(set(metabolomic_influence_edge_tuple).intersection(set(integrated_only)))))

154
987


In [143]:
proteomic_influence_edge_tuple

[('R-HSA-112311', 'R-HSA-110331'),
 ('R-HSA-112311', 'R-HSA-112310'),
 ('R-HSA-112315', 'R-HSA-110331'),
 ('R-HSA-112315', 'R-HSA-112310'),
 ('R-HSA-112315', 'R-HSA-112311'),
 ('R-HSA-1237112', 'R-HSA-112311'),
 ('R-HSA-1237112', 'R-HSA-112315'),
 ('R-HSA-1368082', 'R-HSA-112311'),
 ('R-HSA-1368082', 'R-HSA-112315'),
 ('R-HSA-1368108', 'R-HSA-112311'),
 ('R-HSA-1368108', 'R-HSA-112315'),
 ('R-HSA-1428517', 'R-HSA-110331'),
 ('R-HSA-1428517', 'R-HSA-112310'),
 ('R-HSA-1428517', 'R-HSA-112311'),
 ('R-HSA-1428517', 'R-HSA-112315'),
 ('R-HSA-1428517', 'R-HSA-1237112'),
 ('R-HSA-1428517', 'R-HSA-1368082'),
 ('R-HSA-1428517', 'R-HSA-1368108'),
 ('R-HSA-1483148', 'R-HSA-112311'),
 ('R-HSA-1483148', 'R-HSA-112315'),
 ('R-HSA-1483148', 'R-HSA-1428517'),
 ('R-HSA-1483206', 'R-HSA-110331'),
 ('R-HSA-1483206', 'R-HSA-112310'),
 ('R-HSA-1483206', 'R-HSA-112311'),
 ('R-HSA-1483206', 'R-HSA-112315'),
 ('R-HSA-1483206', 'R-HSA-1237112'),
 ('R-HSA-1483206', 'R-HSA-1368082'),
 ('R-HSA-1483206', 'R-HSA-1

In [142]:
list1 = list(set(metabolomic_influence_edge_tuple).intersection(set(integrated_only)))

counter = 0 

for edges in list1:
    pathway_pair = list(edges)
    if ((pathway_pair[0] in common_pathways) or (pathway_pair[1] not in common_pathways)):
        counter += 1
    if ((pathway_pair[1] in common_pathways) and (pathway_pair[0] not in common_pathways)):
        #print(pathway_pair)
        counter += 1

print(counter)

715


In [134]:
#Pathways where there is only one common pathway 

common_pathways = list(set(proteomic_pathways) & set(metabolomic_pathways))
len(common_pathways)

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if ((pathway_pair[0] in common_pathways) and (pathway_pair[1] not in common_pathways)):
        counter += 1
    if ((pathway_pair[1] in common_pathways) and (pathway_pair[0] not in common_pathways)):
        #print(pathway_pair)
        counter += 1

print(counter)

810


In [131]:
#Pathways where only one is a common pathway and the other is a metabolomics only pathway

common_pathways = list(set(proteomic_pathways) & set(metabolomic_pathways))
len(common_pathways)

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if ((pathway_pair[0] in common_pathways) and (pathway_pair[1] not in common_pathways) and (pathway_pair[1] in metabolomic_pathways)):
        counter += 1
    if ((pathway_pair[1] in common_pathways) and (pathway_pair[0] not in common_pathways) and (pathway_pair[0] in metabolomic_pathways)):
        #print(pathway_pair)
        counter += 1

print(counter)

81


In [127]:
#Pathways where only one is a common pathway and the other is a proteomics only pathway

common_pathways = list(set(proteomic_pathways) & set(metabolomic_pathways))
len(common_pathways)

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if ((pathway_pair[0] in common_pathways) and (pathway_pair[1] not in common_pathways) and (pathway_pair[1] in proteomic_pathways)):
        counter += 1
    if ((pathway_pair[1] in common_pathways) and (pathway_pair[0] not in common_pathways) and (pathway_pair[0] in proteomic_pathways)):
        #print(pathway_pair)
        counter += 1

print(counter)

668


In [128]:
#Pathway pairs where one is part of metabolomics and not proteomics and the other is vice versa

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in metabolomic_pathways and pathway_pair[0] not in proteomic_pathways) and (pathway_pair[1] in proteomic_pathways and pathway_pair[1] not in metabolomic_pathways):
        counter += 1
    if (pathway_pair[0] in proteomic_pathways and pathway_pair[0] not in metabolomic_pathways) and (pathway_pair[1] in metabolomic_pathways and pathway_pair[1] not in proteomic_pathways):
        counter += 1
        
print(counter)

676


In [118]:
#Probably false postives:  96 overall
#Proteomic influenced only (i.e. pairs where both pathways are only in the proteomic dataset)
print(len(list(set(proteomic_only_edge_tuple).intersection(set(integrated_only)))))
#Metabolomic influenced only (i.e. pairs where both pathways are only in the metabolomic dataset)
print(len(list(set(metabolomic_only_edge_tuple).intersection(set(integrated_only)))))

95
1


In [119]:
#Check the significance of the 'false positives' IN THE INTEGRATED DATASET
proteomic_false_pos = list(set(proteomic_only_edge_tuple).intersection(set(integrated_only)))
metabolomic_false_pos = list(set(metabolomic_only_edge_tuple).intersection(set(integrated_only)))

In [120]:
proteomic_false_pos_pval = []
metabolomic_false_pos_pval = []

for edge in proteomic_false_pos:
    if edge in list(p_val_dict.keys()):
        proteomic_false_pos_pval.append(p_val_dict[edge])

for edge in metabolomic_false_pos:
    if edge in list(p_val_dict.keys()):
        metabolomic_false_pos_pval.append(p_val_dict[edge])

proteomic_false_pos_pval[:100]
#len(proteomic_false_pos_pval)

print(metabolomic_false_pos_pval) #[0.000747091124962897] (not that close to threshold)
sum(proteomic_false_pos_pval)/len(proteomic_false_pos_pval)  #average = 0.0045  #close to p < 0.05 threshold

NameError: name 'p_val_dict' is not defined

In [121]:
#Pathway pairs where at least one is one of the 35 new pathways
 
counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in new_pathways) or (pathway_pair[1] in new_pathways):
        #print(pathway_pair[0])
        counter += 1

print(counter)

669


In [137]:
#Pathway pairs where at least one is one of the 35 new pathways
 
counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[1] in new_pathways) and (pathway_pair[0] in new_pathways):
        #print(pathway_pair[0])
        counter += 1

print(counter)

#38+23

14


In [122]:
len(list(set(integrated_edge_tuple).intersection(set(integrated_only))))

2535